In [1]:
import pandas as pd #import pandas library to handle file, database

In [2]:
dataset = pd.read_csv("insurance_pre.csv") #read the file data and store it in dataset variable

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [7]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)
dataset = pd.get_dummies(dataset, drop_first=True)

In [9]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [11]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
print(dataset[boolean_columns])

Index(['sex_male', 'smoker_yes'], dtype='object')
      sex_male  smoker_yes
0            0           1
1            1           0
2            1           0
3            1           0
4            1           0
...        ...         ...
1333         1           0
1334         0           0
1335         0           0
1336         0           0
1337         0           1

[1338 rows x 2 columns]


In [13]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [15]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [17]:
#split input data
independent = dataset [['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [19]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [21]:
dependent = dataset [['charges']] #split output data

In [23]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor #Create a model

param_grid = {'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'], 'n_estimators' : [10,50,100]
             , 'max_features' : [None,'sqrt']}

grid = GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='r2')

#grid.fit(X_train, y_train)

grid.fit(independent, dependent)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': [None, 'sqrt'],
                         'n_estimators': [10, 50, 100]},
             scoring='r2', verbose=3)

In [27]:
re = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(independent) #test the model with input data

from sklearn.metrics import r2_score #Evaluation metrics

r_score = r2_score(dependent, grid_pred)
print("R score value is : ", r_score) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

R score value is :  0.9765555479228759
Best Score: 0.8346885589653879
Best Parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 100}


In [29]:
table = pd.DataFrame.from_dict(re) #convert dictonary format to DataFrame(table)

In [31]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.086063,0.007447,0.005278,0.003163,squared_error,None,10,"{'criterion': 'squared_error', 'max_features':...",0.834731,0.752132,0.842847,0.800545,0.810545,0.808160,0.031975,22
1,0.398520,0.013158,0.012480,0.002433,squared_error,None,50,"{'criterion': 'squared_error', 'max_features':...",0.848699,0.775182,0.848871,0.813364,0.833804,0.823984,0.027644,14
2,0.804262,0.044466,0.023914,0.004681,squared_error,None,100,"{'criterion': 'squared_error', 'max_features':...",0.851407,0.773593,0.858585,0.820537,0.840272,0.828879,0.030482,9
3,0.066346,0.006043,0.009242,0.001870,squared_error,sqrt,10,"{'criterion': 'squared_error', 'max_features':...",0.853608,0.779470,0.837467,0.813765,0.813421,0.819546,0.025133,17
4,0.318777,0.015159,0.015883,0.003317,squared_error,sqrt,50,"{'criterion': 'squared_error', 'max_features':...",0.855230,0.778256,0.865319,0.818918,0.839075,0.831359,0.030846,6
5,0.553617,0.016586,0.027522,0.004432,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.857810,0.782955,0.864752,0.824628,0.843299,0.834689,0.029292,1
6,0.085614,0.008699,0.007517,0.001962,friedman_mse,None,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.839418,0.758931,0.842675,0.809022,0.825642,0.815138,0.030511,20
7,0.409402,0.042804,0.014984,0.004491,friedman_mse,None,50,"{'criterion': 'friedman_mse', 'max_features': ...",0.839749,0.777880,0.856973,0.813938,0.832364,0.824181,0.026957,13
8,0.733230,0.034606,0.024171,0.003981,friedman_mse,None,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.847297,0.771037,0.854586,0.817329,0.839288,0.825907,0.030144,12
9,0.061110,0.001969,0.006990,0.004800,friedman_mse,sqrt,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.846158,0.764328,0.830133,0.809821,0.823842,0.814856,0.027830,21


In [33]:
age = int(input("Enter the age:"))
bmi = float(input("Enter the BMI:"))
children = int(input("Enter the children number:"))
sex = int(input("Enter sex (0 for female and 1 for male):"))
smoker = int(input("Enter 0 for non smoker and 1 for smoker:"))

Enter the age: 26
Enter the BMI: 20
Enter the children number: 0
Enter sex (0 for female and 1 for male): 0
Enter 0 for non smoker and 1 for smoker: 1


In [35]:
Final_prediction =grid.predict([[age,bmi,children,sex,smoker]])
print("Final_prediction of RF: {}".format(Final_prediction))

Final_prediction of RF: [15732.7692592]


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [37]:
import pickle

fileName = "RF_Grid_Insurance.sav"

pickle.dump(grid, open(fileName,'wb'))

In [39]:
load_model = pickle.load(open(fileName,'rb'))

result = load_model.predict([[30,25.899,2,1,1]])
result

C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([18958.8146815])